In [1]:
from torch.utils.data import DataLoader
from lightning.pytorch.loggers import WandbLogger
from TModel.GuitarTokenEmbeddingModel import GuitarTokenEmbeddingModel
from TranscriptionDataset import GuitarTokenDataset
from lightning import Trainer
import torch

# torch.set_float32_matmul_precision('medium')

In [2]:
datasetLocation = "Trainsets/S_Tier_1695428558_mTokens1000_mNoS60.hdf5"
wandbProject = "GuitarEmbeddingModel"
batchSize = 1024
num_workers=12

In [4]:
dataset,pipe = GuitarTokenDataset.getDataPipe(datasetLocation,2,pinMemory=False)
# train_pipe, test_pipe = pipe.random_split(weights={"train": 0.8, "valid": 0.2},seed=42,total_length=len(dataset))

train_dl = DataLoader(dataset=pipe, batch_size=batchSize,num_workers=num_workers)
# test_dl = DataLoader(dataset=test_pipe, batch_size=batchSize,num_workers=num_workers)

In [5]:
model = GuitarTokenEmbeddingModel(vocab_size=dataset.getVocabSize(),cfs=dataset.getCfs())

wandb_logger = WandbLogger(project=wandbProject)
wandb_logger.experiment.config.update(dataset.meta_data)
wandb_logger.experiment.config["batchSize"] = batchSize
trainer = Trainer(
    max_epochs=5,
    # profiler="simple",
    # precision="bf16-mixed",
    logger=wandb_logger,
    # max_time="00:00:05:00"
)

trainer.fit(model=model, train_dataloaders=train_dl)

wandb: Currently logged in as: draguve. Use `wandb login --relogin` to force relogin


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3070 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type      | Params
-----------------------------------------
0 | embeddings | Embedding | 4.5 M 
1 | loss       | SGNSLoss  | 4.5 M 
-----------------------------------------
4.5 M     Trainable params
0         Non-trainable params
4.5 M     Total params
18.190    Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.


In [ ]:
trainer.test(model, dataloaders=test_pipe)